Chapter 2.a.iv.  Operation reductions
=======

**Operation** is the base class for most of the **Expression** classes defined in the **Prove-It** theory systems.  An **Operation** represents *operator(s)* acting upon *operand(s)*.  $1 + 5 - 1$, $1 < k \leq l$, and $x_1 \cdot \ldots \cdot x_n$ are a few examples of expressions that can be represented by classes derived from **Operation**.

There are essentially two kinds of reduction rules specific to **Operation** expressions when the operation itself is to be replaced, rather than just its operator and/or operand(s):
1. *Explicit operation replacement* is performed by replacing an operator with a **Lambda** expression.  In that case, the operation is replaced by the result of applying thislambda mapping to the operands (an internal beta reduction).  For example, $P(x, y)$ with $P:[(a, b) \mapsto a+b]$ becomes $x + y$.
2. *Implicit operation replacement* is performed by replacing an operator with a designated **Literal** operator of an **Operation** class.  In that case, the operation is replaced by one of that derived class rather than the **Operation** class. For example, $P(x, y)$ with $P:+$ becomes $x + y$ with the correct `Add` type rather than a generic **Operation** type.

In [1]:
import proveit
%begin operation_reductions

## Creating operation expressions

Operation expressions come in various forms.  In any of its forms an **Operation** has two sub-expressions representing its *operator(s)* and *operand(s)* respectively.  Each of these may either be an **ExprTuple** or it represents a singular operator/operand.  There are two main types of formatting for an **Operation**, though any formatting rules may be defined by a class that derives from **Operation**.  These are *infix* and *function* formatting.

The default formatting of an **Operation** class is *infix* notation. However, we cannot create an object of type **Operation**:

In [2]:
from proveit import Operation
from proveit import a, b, c
from proveit.numbers import Add

try:
    add_abc_opclass = Operation(Add._operator_, (a, b, c), styles={})
    assert False
except TypeError as e:
    print("Expected error: %s"%str(e))

Expected error: Do not create an object of type Operation; use a derived class (e.g., Function) instead.


We must use a class derived from **Operation**:

In [3]:
class MyOp(Operation):
    def __init__(self, *operands, styles=None):
        Operation.__init__(self, Add._operator_, operands, styles=styles)
    
add_abc_opclass = MyOp(a, b, c)
print("To do this properly and be able to display the expression, the class needs to be "
      "defined in its own module.")

To do this properly and be able to display the expression, the class needs to be defined in its own module.


Note, however, that this is not the proper way to create an expression representing $a+b+c$ because it is not of type `Add`, we just borrowed the **Literal** operator for the `Add` operation called `Add._operator_`.

In [4]:
add_abc_opclass.__class__

__main__.MyOp

Instead, we should use the `Add` constructor.

In [5]:
add_abc = Add(a, b, c)

In [6]:
add_abc.__class__

proveit.numbers.addition.add.Add

When you want *function* formatting, it is best to use the `Function` class that is derived from `Operation`:

In [7]:
from proveit import Function
from proveit import f, g, x, y, z
Function(f, x)

f(x)

In [8]:
Function(f, (x, y, z))

f(x, y, z)

## Operation replacement

We will demonstrate the two types of *operation replacement* in the following examples: explicit and implicit.

### Explicit operation replacement

To start our demonstration, let us define a **Lambda** expression which maps a variable $f$ to an expression in which $f$ is used as an operator.

In [9]:
from proveit import Function, Lambda
from proveit.logic import Equals
from proveit import a, b, c, f, n, x, y, z
ab_substitution = Lambda(f, Equals(Function(f, a), Function(f, b)))

Now we just need to define a **Lambda** expression for replacing $f$.

In [10]:
from proveit.numbers import Add, Mult
add_y_then_mult_z = Lambda(x, Mult(Add(x, y), z))

In [11]:
ab_substitution.apply(add_y_then_mult_z)

((a + y) * z) = ((b + y) * z)

Note that the $f(a)$ and $f(b)$ operations themselves were replaced according to the `add_y_then_mult_z` lambda mapping applied to $a$ and $b$ respectively.  Let's show an example with multiple operands.

In [12]:
ab_commutation = Lambda(f, Equals(Function(f, [a, b]), Function(f, [b, a])))

In [13]:
add_then_mult_z = Lambda((x, y), Mult(Add(x, y), z))

In [14]:
ab_commutation.apply(add_then_mult_z)

((a + b) * z) = ((b + a) * z)

### Implicit operation replacement

It is typical to derive classes from **Operation** that have a single corresponding *operator* that is a **Literal**.  This operator should be defined as the `_operator_` attribute of the derived class.  This indicates to **Prove-It** that this specific operator is tied to that specific class.  This knowlege is used in *implicit operation replacement*.  When an operator is replaced by `_operator_` of a particular **Operation** class, the **Operation** is reconstucted to by of that particular type.  For example.

In [15]:
a_add_b_commutation = ab_commutation.apply(Add._operator_)

Is created using the proper `Add` class for constructing the left and right sides of the equation.

In [16]:
a_add_b_commutation.lhs.__class__

proveit.numbers.addition.add.Add

In [17]:
a_add_b_commutation.rhs.__class__

proveit.numbers.addition.add.Add

We can do the same using `Mult._operator_`:

In [18]:
a_mult_b_commutation = ab_commutation.apply(Mult._operator_)

In [19]:
a_mult_b_commutation.lhs.__class__

proveit.numbers.multiplication.mult.Mult

In [20]:
%end operation_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>